In [8]:
import advent
import numpy as np
data = [int(i) for i in advent.get_lines(16)[0]]
print(data[:10])

[5, 9, 7, 9, 3, 5, 1, 3, 5, 1]


In [2]:
last_digit = lambda d: d % 10 if d >= 0 else (-d) % 10

def apply_pattern(input, pattern):
    # input is list of length N
    # pattern is list of length M (already repeated if needed)
    # extend pattern by calling np.tile on it so it's longer than input,
    #   then clipping to same length as input, and skipping the very first element of the pattern
    # then return dot product
    tile = (len(input) // len(pattern)) + 1  # +1 so we can always skip the first element
    pattern_extended = np.tile(pattern, tile)[1:len(input)+1]
    res = np.dot(input, pattern_extended)
    return last_digit(res)

In [ ]:
def apply_full_pattern(input, base_pattern=np.array([0, 1, 0, -1])):
    output = np.zeros(len(input))
    for ix, _ in enumerate(input):
        pattern = np.repeat(base_pattern, ix+1)
        output[ix] = apply_pattern(input, pattern)
    return output

test_input = np.array([1, 2, 3, 4, 5, 6, 7, 8])
print(apply_full_pattern(test_input)) # should be 48226158

[4. 8. 2. 2. 6. 1. 5. 8.]


In [18]:
def apply_full_pattern_n_times(input, n=100, base_pattern=np.array([0, 1, 0, -1])):
    for _ in range(n):
        input = apply_full_pattern(input, base_pattern)
    return input

test_input = np.array([int(i) for i in '80871224585914546619083218645595'])
print(''.join(str(int(i)) for i in apply_full_pattern_n_times(test_input))) # should start with 24176176

24176176480919046114038763195595


In [19]:
print(''.join(str(int(i)) for i in apply_full_pattern_n_times(data, n=100)[:8]))  # assignments only wants the first 8 digits

49254779


In [ ]:
# Part 2
long_data = np.tile(data, 10000)
offset = int(''.join(str(i) for i in data[:7]))

# Tries to use numpy for everything to eliminate a for loop
def fft_faster(data):
    # We are just going to make the assumption the pattern is always [0, 1, 0, -1], no need for a parameter
    base_pattern = np.array([0, 1, 0, -1])
    size = len(data)
    result = np.zeros(size)

    for n in range(1, size+1):
        applied_pattern = np.floor((np.arange(1, size+1) % (4*n)) / n).astype(int)
        applied_pattern = base_pattern[applied_pattern]
        result[n-1] = last_digit(np.dot(data, applied_pattern))
    return result
print(fft_faster([int(i) for i in '12345678']))
# Still too slow when we apply it on long_data

[4. 8. 2. 2. 6. 1. 5. 8.]


In [ ]:
# Attempt 2

# After sleeping on it for a few days (or years...?) I checked the offset, and noticed it was
# more than 90% of the length of the data (5979351 / 6500000 ~= 0.92)
# Specifically in the second half of the list, where j=5979351, a_ij = a_(i-1)j + a_(i-1)(j+1) + ...
# And we can just ignore the entire pattern since it's just ones and just use cumsum
# we don't use last_digit function since we know the sum will never be negative we can just do % 10

data_from_offset = long_data[offset:]

for _ in range(100):
    # Calculate the entire 'second half of the list' (from offset only) by just taking the cumsum
    # And then the last digit of each element
    # Do this a hundred times
    data_from_offset = np.cumsum(data_from_offset[::-1])[::-1] % 10

print(''.join(str(c) for c in data_from_offset[:8]))


55078585
